<a href="https://colab.research.google.com/github/JeongSoong/Machine_learning/blob/main/%EB%AC%B8%EC%A0%95%EC%88%98_201911047_14%EC%A3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
IMG_SIZE = 300

def preprocess(image, label):
    image1 = tf.image.resize(image, size=(300, 300))
    image2 = image1/255
    return image2, label

In [ ]:
def solution_model():
    (ds_train, ds_validation), ds_info = tfds.load(
        name='beans',
        split=['train', 'validation'],
        as_supervised=True,
        with_info=True
    )

    BATCH_SIZE = 32
    ds_train = ds_train.map(preprocess).cache().shuffle(
        ds_info.splits['train'].num_examples).batch(BATCH_SIZE).prefetch(
        tf.data.experimental.AUTOTUNE)
    ds_validation = ds_validation.map(preprocess).batch(BATCH_SIZE).cache(
    ).prefetch(tf.data.experimental.AUTOTUNE)

    model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        # 2D -> 1D로 변환을 위해 Flatten을 사용합니다.
        Flatten(),
        # 과적합 방지를 위해 Dropout을 적용합니다. 학습 시에 큰 노드 50%만 사용
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(3, activation='softmax'),
    ])

    model.compile(
        optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc']
    )
    epochs = 10
    model.fit(
        ds_train,
        validation_data=(ds_validation),
        epochs=epochs
    )
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel_3.h5")

In [ ]:
import matplotlib.pylab as plt
import tensorflow_datasets as tfds

datasets, info = tfds.load(name = 'beans', with_info=True,
                           as_supervised = True,
                           split = ['train', 'test', 'validation'])

In [ ]:
train, info_train = tfds.load(name = 'beans',
                              with_info = True, split = 'test'
                              )

In [ ]:
tfds.show_examples(train, info_train)

3번유형(인공위성에서 촬영한 허리케인 분류)

In [ ]:
import urllib
import zipfile

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')

with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rescale=1/255.,
                               horizontal_flip=True,
                               rotation_range=35,
                               zoom_range=0.2
                               )
train_dir = '/content/train'
train_gen = image_gen.flow_from_directory(train_dir,
                                        target_size=(300, 300),
                                        batch_size=128,
                                        class_mode='binary'
                                        )

In [ ]:
class_labels = ['damage', 'no_damage']
batch=next(train_gen)
images, labels = batch[0], batch[1]

plt.figure(figsize=(16,8))
for i in range(32):

    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis('off')
plt.tight_layout()
plt.show()

Time Series Question(시계열 데이터, LSTM)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)
    
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS
    
    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    
    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
    N_PAST = 10  # DO NOT CHANGE THIS
    N_FUTURE = 10  # DO NOT CHANGE THIS
    SHIFT = 1  # DO NOT CHANGE THIS

    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    # Code to define your model.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
        tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True)),        
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),                  
        tf.keras.layers.Dense(N_FEATURES) # 출력층
    ])
    
    # Code to train and compile the model
    checkpoint_path='model/my_checkpoint.ckpt'

    checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1,
                             )
    # Code to train and compile the model
    optimizer =  tf.keras.optimizers.Adam(0.00003)  # YOUR CODE HERE
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])

    model.fit(train_set, validation_data=(valid_set), epochs=150, callbacks=[checkpoint])

    model.load_weights(checkpoint_path)

    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("model.h5")

In [ ]:
import zipfile
import pandas as pd

In [ ]:
local_zip="/content/HouseholdElectricPowerConsumption.zip"
zip_ref = zipfile.Zipfile(local_zip, 'r')
zip_ref.extractall('/content')

In [ ]:
 df = pd.read_csv("/content/household_power_consumption.txt", sep=',',
                  parse_dates={'dt': ['Date', 'Time']},
                     infer_datetime_format=True,
                     low_memory=False, na_values=['nan','?'],
                     index_col='dt')
 
 df.head()

In [ ]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf

# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the un windowed time series dataset to a
# windowed dataset so as to prepare it for training and validation.
# A window of features are constructed by shifting the window's starting
# position forward, one at a time (indicated by shift=1).
# For a window of 'n_past' number of observations of all time indexed variables in
# the dataset, the target for the window is the next 'n_future' number
# of observations of these variables, after the end of the window.
# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    # This line converts the dataset into a windowed dataset where a
    # window consists of both the observations to be included as features
    # and the targets.
    # Don't change the shift parameter. The test windows are
    # created with the specified shift and hence it might affect your
    # scores. Calculate the window size so that based on
    # the past 24 observations
    # (observations at time steps t=1,t=2,...t=24) of the 7 variables
    # in the dataset, you predict the next 24 observations
    # (observations at time steps t=25,t=26....t=48) of the 7 variables
    # of the dataset.
    # Hint: Each window should include both the past observations and
    # the future observations which are to be predicted. Calculate the
    # window size based on n_past and n_future.
    ds = ds.window(size=  # YOUR CODE HERE,
                   shift = shift,
                           drop_remainder = True)
    # This line converts the windowed dataset into a tensorflow dataset.
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    # Now each window in the dataset has n_past and n_future observations.
    # This line maps each window to the form (n_past observations,
    # n_future observations) in the format needed for training the model.
    # Note: You can use a lambda function to map each window in the
    # dataset to it's respective (features, targets).
    ds = ds.map(
        # YOUR CODE HERE
    )
    return ds.batch(batch_size).prefetch(1)


# This function loads the data from csv file, normalizes the data and
# splits the dataset into train and validation data. It also uses the
# 'windowed_dataset' to split the data into windows of observations and
# targets. Finally it defines, compiles and trains a neural network.
# This function returns the trained model.
#
# COMPLETE THE CODE IN THE FOLLOWING FUNCTION.
def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    # Reads the dataset from the csv.
    df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
    # Number of features in the dataset. We use all features as predictors to
    # predict all features at future time steps.
    N_FEATURES = len(df.columns)
    # Normalizes the data
    data = df.values
    split_time = int(len(data) * 0.5)
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    # Splits the data into training and validation sets.
    x_train = data[:split_time]
    x_valid = data[split_time:]
    # DO NOT CHANGE 'BATCH_SIZE' IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 24  # DO NOT CHANGE THIS
    # Number of future time steps which are to be predicted.
    N_FUTURE = 24  # DO NOT CHANGE THIS
    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS
    # Code to create windowed train and validation datasets.
    # Complete the code in windowed_dataset.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    # Code to define your model.
    model = tf.keras.models.Sequential([
        # ADD YOUR LAYERS HERE.
        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # Whatever your first layer is, the input shape will be
        # (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7)
        # The model must have an output shape of
        # (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7).
        # Make sure that there are N_FEATURES = 7 neurons in the final dense
        # layer since the model predicts 7 features.
        
        Conv1D(filters=32,     kernel_size=3,       padding="causal",
            activation="relu",       input_shape=[N_PAST, 7],
            ),
         Bidirectional(LSTM(32, return_sequences=True)),
         Bidirectional(LSTM(32, return_sequences=True)),
         Dense(32, activation="relu"),
         Dense(16, activation="relu"),
         Dense(N_FEATURES)

        tf.keras.layers.Dense(N_FEATURES)
    ])
    # Code to train and compile the model
    optimizer =  # YOUR CODE HERE
    model.compile(
        # YOUR CODE HERE
    )
    model.fit(
        # YOUR CODE HERE
    )
    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("model.h5")

# 새 섹션

In [ ]:
##데이터 확인하기

import urllib
import pandas as pd
import tensorflow as tf

In [ ]:
url = "https://www.dropbox.com/s/eduk281didil1km/Weekly_U.S.Diesel_Retail_Prices.csv?dl=1"
urllib.request.urlretrieve(url, 'Weekly_U.S.Diesel_Retail_Prices.csv')

In [ ]:
pd.read_csv("Weekly_U.S.Diesel_Retail_Prices.csv", infer_datetime_format=True,
            index_col='Week of', header=0)